In [7]:
### National Snow and Ice Data Center Marginal Ice Zone Analysis

### National Snow and Ice Data Center Marginal Ice Zone Analysis

## Overview

The National Snow and Ice Data Center receives an operational Marginal Ice Zone (MIZ) product from the National Ice Center (USNIC) daily.  This project aims to compare the product generated by the USNIC with an automated passive microwave model.

In [ ]:
from modules.io import download_cdr_miz_range, download_nic_miz_range
from modules.compare import nic_file_to_grid, cdr_file_to_grid
from modules.display import plot_grid
import datetime
import numpy as np

import os

start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2020, 2, 1)

# Each grid cell in the cdr represents 25 km x 25 km
# TODO - derive this dynamically from the netcdf
grid_cell_area = 25 * 25

print("Pulling files down from %s to %s" % (start, end))
cdr_files = download_cdr_miz_range(start, end)
nic_files = download_nic_miz_range(start, end)

# For now, let's just loop through cdf_files and nic_files zipped
# If the length isn't the same (or if they are out of order) then
# this should be a red flag that we aren't comparing the same dates
# TODO - instead of looping through zip(cdr_files, nic_files),
# loop through the desired date ranges and generate the file
# names for the dates

# At least we can perform this basic check
assert len(cdr_files) == len(nic_files)

for cdr_file, nic_file in zip(cdr_files, nic_files):
    
    print(f"Comparing {os.path.basename(cdr_file)} and {os.path.basename(nic_file)}")
    
    # Load the files into memory as numpy arrays
    # This involves a rasterization of the nic grid which may be
    # memory-intensive
    nic_grid = nic_file_to_grid(nic_file)
    cdr_grid = cdr_file_to_grid(cdr_file)
    
    # Since we are mainly interested in the MIZ, let's filter so we only get 18%-80% coverage in our grids
    low_thresh = .18
    high_thresh = .8
    nic_grid = np.ma.masked_where((nic_grid < low_thresh) | (nic_grid >= high_thresh), nic_grid)
    cdr_grid = np.ma.masked_where((cdr_grid < low_thresh) | (cdr_grid >= high_thresh), cdr_grid)
    
    nic_miz_area = nic_grid.count() * grid_cell_area
    cdr_miz_area = cdr_grid.count()  * grid_cell_area
    
    print(f"nic; {nic_miz_area} sq km, cdr; {cdr_miz_area} sq km, difference; {nic_miz_area - cdr_miz_area} sq km")
    
    # For now just plot
    plot_grid(nic_grid)
    plot_grid(cdr_grid)
